<h1>ETL</h1>

In [1]:
import pandas as pd
import json
from datetime import datetime

<h3>Dataset "user_items"</h3>

In [13]:
users_items = r"..\Datasets\users_items\australian_users_items.json"

# Abrimos el archivo JSON en modo lectura:
with open(users_items, 'r', encoding='utf-8') as ui_lectura:
    lineas_json = ui_lectura.readlines()

# Convertimos las lineas JSON en objetos Python:
datos_json = [eval(line.strip()) for line in lineas_json]

# Cargamos el archivo en un DataFrame:
df_ui = pd.DataFrame(datos_json)
df_ui.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [14]:
# Eliminamos las columnas innecesarias:
del df_ui["items_count"]
del df_ui["steam_id"]
del df_ui["user_url"]

In [15]:
df_ui.head()

,user_id,items
0,76561197970982479,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


Tratamiento de la columna "items"

In [16]:
# Aplicamos una función lambda para desanidar las claves que utilizaremos y sus respectivos valores:
df_ui["playtime"] = df_ui["items"].apply(lambda x: x[0].get("playtime_forever") if len(x) > 0 else 0)
df_ui["item_id"] = df_ui["items"].apply(lambda x: x[0].get("item_id") if len(x) > 0 else 0)

# Convertimos el tipo de cada registro en "int":
df_ui["playtime"] = df_ui["playtime"].astype(int)
df_ui["item_id"] = df_ui["item_id"].astype(int)

In [17]:
df_ui.dtypes

user_id     object
items       object
playtime     int32
item_id      int32
dtype: object

In [18]:
# Eliminamos la columna "items":
del df_ui["items"]

In [19]:
df_ui.head()

,user_id,playtime,item_id
0,76561197970982479,6,10
1,js41637,0,10
2,evcentric,923,1200
3,Riot-Punch,0,10
4,doctr,1131,300


In [20]:
# Eliminamos los registros duplicados
df_ui = df_ui.drop_duplicates()

Tratamiento de la columna "item_id"

In [21]:
# Eliminaremos todos los registros cuyo juego (item_id), no se encuentre en el registro de juegos de Dataset "steam_games":

# Cargamos el archivo de "steam_games" ya transformado en "ETL_steam_games.ipynb":
games_df = pd.read_csv(r"../Archivos Producidos/games.csv")

In [22]:
# Verificamos qué item_id en "df_ui" no están en "games_df":
items_a_eliminar = df_ui[~df_ui['item_id'].isin(games_df['item_id'])]

# Eliminamos los registros correspondientes de "df_ui":
df_ui = df_ui[df_ui['item_id'].isin(games_df['item_id'])]

Capitalizamos y quitamos espacios en la columnas "user_id"

In [23]:
df_ui["user_id"] = df_ui["user_id"].astype(str).str.capitalize().str.strip()

In [24]:
df_ui[["user_id"]].head()

,user_id
0,76561197970982479
1,Js41637
2,Evcentric
3,Riot-punch
4,Doctr


Guardado como CSV

In [25]:
df_ui.to_csv(r"..\Archivos Producidos\users.csv", index=False)